# Notebook with some interesting Tesla stats

In [47]:
import datetime
import pandas as pd
import numpy as np

## Loading the .csv files into a dataframe

In [48]:
charges_df = pd.read_csv("./charges.csv", parse_dates = ['ChargeStartedAt','ChargeEndedAt'])
charges_df.head(5)

,ChargeId,ChargeStartedAt,ChargeEndedAt,ChargeDurationMin,ChargeLocation,ChargeSavedLocation,ChargeLatitude,ChargeLongitude,ChargeSupercharger,ChargeOdometer,ChargeEnergyAdded,ChargeEnergyUsed,ChargeRatedRangeAdded,ChargeStartingBattery,ChargeEndingBattery,ChargeCost,ChargeCostPerKwh
0,1,2024-01-05 10:12:00,2024-01-05 10:32:00,19,"Nellinger Straße 22, 89188 Merklingen, Germany",SuperCharger,48.516330,9.757481,True,6170.90,28.43,31.20,190.69,35,71,14.26,0.45
1,2,2024-01-06 11:20:00,2024-01-06 12:11:00,50,"Porscheplatz 5, 70435 Stuttgart, Germany",SuperCharger,48.834312,9.152498,False,6357.38,8.88,8.97,59.45,32,44,NaN,NaN
2,3,2024-01-06 13:04:00,2024-01-06 13:37:00,33,"Hohenaspen 40, 69231 Rauenberg, Germany",SuperCharger,49.273937,8.695297,True,6447.10,49.54,54.47,332.28,17,80,24.30,0.44
3,4,2024-01-06 16:37:00,2024-01-06 17:05:00,28,"An Der Patria 15a, 50374 Erftstadt, Germany",SuperCharger,50.796574,6.779480,True,6706.38,50.23,53.54,336.61,11,75,27.03,0.50
4,5,2024-01-06 19:05:00,2024-01-06 23:53:00,287,"Mertens En Torfsstraat 36, 2018 Antwerpen, Ant...",DMA03,51.208317,4.406777,False,6909.28,53.14,54.44,355.92,23,90,19.05,0.34


In [49]:
drives_df = pd.read_csv("./drives.csv", parse_dates = ['DriveStartedAt','DriveEndedAt'])
drives_df.head(5)

,DriveId,DriveStartedAt,DriveEndedAt,DriveTag,DriveDuration,DriveStartingLocation,DriveStartingSavedLocation,DriveStartingLatitude,DriveStartingLongitude,DriveStartingOdometer,...,DriveAverageInsideTemperature,DriveAverageOutsideTemperature,DriveAverageSpeed,DriveMaxSpeed,DriveRatedRangeUsed,DriveIdealRangeUsed,DriveDistance,DriveNonAutopilotDistance,DriveTotalEnergyUsed,DriveAverageEnergyUsed
0,1,2024-01-05 08:50:00,2024-01-05 10:12:00,NaN,81,"Victor-Gollancz-Straße 1, 76137 Karlsruhe, Ger...",Unknown,48.993626,8.398090,6038.98,...,17.69,6.38,101,134,192.61,192.61,131.90,0,35.90,272
1,2,2024-01-05 10:32:00,2024-01-05 11:01:00,NaN,29,"Nellinger Straße 22, 89188 Merklingen, Germany",Unknown,48.516330,9.757481,6170.90,...,17.97,5.07,100,130,49.81,49.81,47.31,0,7.43,157
2,3,2024-01-05 11:10:00,2024-01-05 11:15:00,NaN,5,"Legoland-Allee 3, 89312 Günzburg, Germany",Unknown,48.425137,10.311680,6218.22,...,17.62,6.00,31,68,1.82,1.82,2.56,0,0.27,105
3,4,2024-01-05 18:18:00,2024-01-05 18:27:00,NaN,9,"B 16, 89312 Günzburg, Germany",Unknown,48.422844,10.298338,6220.78,...,15.96,5.50,23,51,6.02,6.02,3.25,0,0.90,277
4,5,2024-01-06 09:36:00,2024-01-06 10:59:00,NaN,83,"Legoland-Allee 3, 89312 Günzburg, Germany",Unknown,48.423740,10.309638,6224.03,...,17.64,4.77,100,121,162.70,162.70,133.29,0,24.26,182


## Connecting to Hopsworks

In [50]:
# import hopsworks

# project = hopsworks.login()

# fs = project.get_feature_store()


import hopsworks
conn = hopsworks.connection(
    host="demo-37.cloud.hopsworks.ai",                                # DNS of your Feature Store instance
    hostname_verification=False,                     # Disable for self-signed certificates
    api_key_value="QCQy18U7xkr4IZZ9.PP9a03HsgIeLUneCbsreQ1chA96hqHWihGcm9JK8nzAjKsmoPpq4r1xUVbLuBFQ0"          # Feature store API key value 
)
project = conn.get_project('TeslaProject') # specify your project name
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


## Creating the Feature Groups

In [54]:
# Define Charges Feature Group
charges_fg = fs.get_or_create_feature_group(
    name = "tesla_charges",
    version = 1,
    primary_key = ["ChargeId"],
    partition_key = ["ChargeLocation"],   
    description = "all the tesla charges",
    event_time = 'ChargeStartedAt',
    online_enabled = True,
    statistics_config = {"enabled": True, "histograms": True, "correlations": True, "exact_uniqueness": False},
)   

# Insert data frame into Feature Group
charges_fg.insert(charges_df)





Uploading Dataframe: 100.00% |██████████| Rows 46/46 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: tesla_charges_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://demo-37.cloud.hopsworks.ai/p/122/jobs/named/tesla_charges_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2848e07a0>, None)

In [53]:
# Define Drives Feature Group
drives_fg = fs.get_or_create_feature_group(
    name = "tesla_drives",
    version = 1,
    primary_key = ["DriveId"],
    partition_key = ["DriveStartingSavedLocation"],   
    description = "all the tesla drives",
    event_time = 'DriveStartedAt',
    online_enabled = True,
    statistics_config = {"enabled": True, "histograms": True, "correlations": True, "exact_uniqueness": False}
)   

# Insert data frame into Feature Group
drives_fg.insert(drives_df)





Uploading Dataframe: 100.00% |██████████| Rows 204/204 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: tesla_drives_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://demo-37.cloud.hopsworks.ai/p/122/jobs/named/tesla_drives_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2848e0c20>, None)